In [12]:
## Dependencies
import pandas as pd
import numpy as np
import xarray as xr

pd.set_option('display.max_columns', None)

from covariate_functions import *

# Load data

In [2]:
## Get lat-lon coordinates of each experimental unit
plots = pd.read_csv('../Input_data/all_lat_lon.csv')

## Assign each plot a unique name
plots['unique_nm'] = plots[['Site', 'UnitID', 'PlotID']].T.agg('_'.join)

## Drop ones without geographic coordinates
plots.drop(plots.loc[plots['Latitude'].isnull()].index, inplace = True)
plots.reset_index(inplace=True, drop = True)
plots

## Drop sites we don't use
plots = plots.loc[plots['Site'].isin(['SNFDPN', 'UPFU'])==False]
plots['Site'].unique()

array(['Sequoia_FFS', 'Sequoia_Tharp', 'STEF-VDT', 'Teakettle',
       'Blodgett', 'UPFUb', 'LaTour'], dtype=object)

In [3]:
# Load cwd by climate year
cwd_by_yr = xr.open_dataset('../Outputs/cwd_by_waterYr.nc', decode_times=False)
cwd_by_yr = cwd_by_yr.rename({'latitude':'lat', 'longitude':'lon'})
cwd_by_yr

<xarray.Dataset> Size: 10MB
Dimensions:         (lon: 248, lat: 232, time: 45)
Coordinates:
  * lon             (lon) float64 2kB -124.4 -124.4 -124.4 ... -114.2 -114.1
  * lat             (lat) float64 2kB 42.06 42.02 41.98 ... 32.52 32.48 32.44
  * time            (time) float64 360B 10.0 11.0 12.0 13.0 ... 52.0 53.0 54.0
Data variables:
    cwd_by_waterYr  (time, lat, lon) float32 10MB ...
    crs             int32 4B ...
Attributes:
    Conventions:  CF-1.4
    created_by:   R packages ncdf4 and terra (version 1.7-71)
    date:         2025-08-01 10:43:42

# Get CWD by unit by year

In [4]:
# Specify time periods of interest
start_yr = 1980
end_yr = 2024

winter_months = [10, 11, 12, 1, 2] # a climate year runs Oct-Sept
summer_months = [6, 7, 8, 9]

In [5]:
# Find closest grid cell to each plot
cwd_plot_year = find_plot_gridcells(cwd_by_yr, plots)

# Add a year column
cwd_plot_year['year'] = cwd_plot_year['time'] + 1970

cwd_plot_year

,lon,lat,time,cwd_by_waterYr,crs,Site,UnitID,PlotID,unique_nm,year
0,-118.770833,36.562500,10.0,577.000000,-2147483647,Sequoia_FFS,FFS7CONTROL,FFS7CONTROL,Sequoia_FFS_FFS7CONTROL_FFS7CONTROL,1980.0
1,-118.770833,36.562500,11.0,824.900024,-2147483647,Sequoia_FFS,FFS7CONTROL,FFS7CONTROL,Sequoia_FFS_FFS7CONTROL_FFS7CONTROL,1981.0
2,-118.770833,36.562500,12.0,518.599976,-2147483647,Sequoia_FFS,FFS7CONTROL,FFS7CONTROL,Sequoia_FFS_FFS7CONTROL_FFS7CONTROL,1982.0
3,-118.770833,36.562500,13.0,477.899994,-2147483647,Sequoia_FFS,FFS7CONTROL,FFS7CONTROL,Sequoia_FFS_FFS7CONTROL_FFS7CONTROL,1983.0
4,-118.770833,36.562500,14.0,824.400024,-2147483647,Sequoia_FFS,FFS7CONTROL,FFS7CONTROL,Sequoia_FFS_FFS7CONTROL_FFS7CONTROL,1984.0
...,...,...,...,...,...,...,...,...,...,...
25240,-121.645833,40.604167,50.0,578.200012,-2147483647,LaTour,221,221,LaTour_221_221,2020.0
25241,-121.645833,40.604167,51.0,690.299988,-2147483647,LaTour,221,221,LaTour_221_221,2021.0
25242,-121.645833,40.604167,52.0,505.500000,-2147483647,LaTour,221,221,LaTour_221_221,2022.0
25243,-121.645833,40.604167,53.0,406.399994,-2147483647,LaTour,221,221,LaTour_221_221,2023.0


In [6]:
# Group plots by unit so that sites with subplots are averaged at the unit level
cwd_unit_year = cwd_plot_year.loc[:,['cwd_by_waterYr', 'Site', 'UnitID', 'year']].groupby(['UnitID', 'Site', 'year'], 
                                                                                          as_index=False).agg('mean')

cwd_unit_year

,UnitID,Site,year,cwd_by_waterYr
0,1,LaTour,1980.0,346.500000
1,1,LaTour,1981.0,457.200012
2,1,LaTour,1982.0,302.399994
3,1,LaTour,1983.0,332.700012
4,1,LaTour,1984.0,449.299988
...,...,...,...,...
13000,WRD 20-9 T,UPFUb,2020.0,637.000000
13001,WRD 20-9 T,UPFUb,2021.0,710.700012
13002,WRD 20-9 T,UPFUb,2022.0,543.599976
13003,WRD 20-9 T,UPFUb,2023.0,423.399994


In [7]:
# Group by site
cwd_site_year = cwd_unit_year.loc[:,['cwd_by_waterYr', 'Site', 'year']].groupby(['Site', 'year'], as_index=False).agg('mean')
cwd_site_year

,Site,year,cwd_by_waterYr
0,Blodgett,1980.0,391.164154
1,Blodgett,1981.0,591.950684
2,Blodgett,1982.0,349.535767
3,Blodgett,1983.0,366.415131
4,Blodgett,1984.0,512.248230
...,...,...,...
310,UPFUb,2020.0,658.678589
311,UPFUb,2021.0,727.775024
312,UPFUb,2022.0,566.317871
313,UPFUb,2023.0,449.746429


In [8]:
def get_tree_cwd_timeseries(site, cwd_site_year = cwd_site_year):
    '''Assign a mean CWD to each tree's measurement interval'''
    
    ## Open tree-level data for site
    pft_df = pd.read_csv('../../State_space_growth_models/Input_data/{}/pft_df.csv'.format(site))
    
    ## Instantiate a CWD column
    pft_df['CWD'] = np.nan
    
    ## Loop through each individual tree
    for tree in pft_df['TreeID'].unique():
        tree_df = pft_df.loc[pft_df['TreeID']==tree]
        cwd_tree = []
        
        # Loop through all observations for tree
        for c in tree_df['counter']:
            
            ## Get start and end year of the measurement interval
            st_yr = tree_df.loc[tree_df['counter']==c, 'start_year'].iloc[0]
            end_yr = tree_df.loc[tree_df['counter']==c, 'end_year'].iloc[0]
            
            ## CWD is null at the starting time point
            if c == 1:
                cwd_mean = np.nan
                
            ## For the first measurement interval, CWD is the average of annual values for the time period inclusive of the first year
            elif c == 2:
                cwd_mean = cwd_site_year.loc[(cwd_site_year['Site']==site) & (cwd_site_year['year']>=st_yr) & (cwd_site_year['year']<=end_yr), 'cwd_by_waterYr'].mean()
                
            ## For all subsequent intervals, CWD is the average of annual values for the time period exclusive of the first year
            elif c > 2:
                cwd_mean = cwd_site_year.loc[(cwd_site_year['Site']==site) & (cwd_site_year['year']>st_yr) & (cwd_site_year['year']<=end_yr), 'cwd_by_waterYr'].mean()
                
            cwd_tree = cwd_tree + [cwd_mean]
        
        ## Reset the starting CWD to the CWD of the first measurement
        cwd_tree[0] = cwd_tree[1]
            
        ## Add CWD time series to data table
        pft_df.loc[pft_df['TreeID']==tree, 'CWD'] = cwd_tree
    
    assert len(pft_df.loc[pft_df['CWD'].isnull()])==0

    return pft_df

In [9]:
def format_cwd_for_ssm(pft_df_w_cwd):
    
    ## Loop through each tree; make a list of all years associated with each tree
    treeid = []
    year = []
    
    for tree in pft_df_w_cwd['TreeID'].unique():
        tree_df = pft_df_w_cwd.loc[pft_df_w_cwd['TreeID']==tree]
        tree_yrs = np.arange(tree_df['timeseries_year'].min(), tree_df['timeseries_year'].max()+1).tolist()
        year = year + tree_yrs
        treeid = treeid + np.repeat(tree, len(tree_yrs)).tolist()

    ## Make a dataframe with TreeID and Year as columns
    cwd_for_ss = pd.DataFrame({'TreeID':treeid, 'timeseries_year':year})

    ## Merge in CWD values at the beginning and end of each measurement interval
    cwd_for_ss = pd.merge(cwd_for_ss, 
                          pft_df_w_cwd.loc[:,["TreeID", "timeseries_year", "CWD"]], 
                          how='left',
                          on=['TreeID','timeseries_year'])

    ## Check that we didn't drop any measured DBH records
    assert len(cwd_for_ss.loc[cwd_for_ss['CWD'].notnull()]==len(pft_df_w_cwd))

    ## Backfill CWD values for the empty years
    cwd_for_ss['CWD'] = cwd_for_ss['CWD'].bfill()

    # format cwd timeseries so that each row is a tree (should match obs_matrix and year_matrix order)
    cwd_for_ss = pd.pivot_table(values='CWD',index='TreeID',columns='timeseries_year',aggfunc='median', data=cwd_for_ss)
    
    return cwd_for_ss

In [10]:
## Loop through sites; save tree-level CWD timeseries for each site
for site in ['Blodgett']: #'LaTour', 'Sequoia_FFS', 'Sequoia_Tharp', 'STEF-VDT', 'Teakettle', 'UPFUb']:
    temp = get_tree_cwd_timeseries(site)
    temp2 = format_cwd_for_ssm(temp)
    temp2.to_csv('../../State_space_growth_models/Input_data/{}/cwd_tree_year.csv'.format(site))